# Black Box function
- Represents a generic function that we are trying to optimize
- In this case, we use 5 inputs each representing an experimental control

In [5]:
def black_box_function(a, b, c, d, e):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -a * (2*b)**2 - (c - (1*d))**2 * (2*e)**2 + 1

### Import Necessary Package

In [6]:
from bayes_opt import BayesianOptimization

### Set boundaries for input parameters
- These are allowable entries for experimental control variables
- In this case, we have arbitrary values selected for each input

In [7]:
pbounds = {'a': (2, 4), 'b': (-3, 3), 'c': (-8, 3), 'd': (0, 5), 'e': (-2, 2)}

In [8]:
optimizer = BayesianOptimization(
    f=black_box_function, # sets function for optimization to our defined function above
    pbounds=pbounds,# sets the bounds for each of the parameters to above set values
    verbose=2,# verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [9]:
optimizer.maximize(
    init_points=5,
    n_iter=3, 
)

|   iter    |  target   |     a     |     b     |     c     |     d     |     e     |
-------------------------------------------------------------------------------------
| 1         | -741.1    | 2.834     | 1.322     | -7.999    | 1.512     | -1.413    |
| 2         | -33.65    | 2.185     | -1.882    | -4.199    | 1.984     | 0.1553    |
| 3         | -1.483e+0 | 2.838     | 1.111     | -5.751    | 4.391     | -1.89     |
| 4         | -40.41    | 3.341     | -0.4962   | -1.854    | 0.7019    | -1.208    |
| 5         | -695.0    | 3.601     | 2.81      | -4.552    | 3.462     | 1.506     |
| 6         | -55.57    | 2.521     | -2.34     | -2.395    | 0.4218    | 0.2083    |
| 7         | -56.68    | 2.073     | -2.619    | -0.02991  | 3.352     | -0.1322   |
| 8         | -9.273    | 2.673     | 0.259     | 1.779     | 0.7848    | 1.554     |


The best combination of parameters and target value found can be accessed via the property `bo.max`.

In [10]:
print(optimizer.max)

{'target': -9.272902545063692, 'params': {'a': 2.673141578780119, 'b': 0.259002547417742, 'c': 1.7792527885611076, 'd': 0.7847833439795421, 'e': 1.5542039374035834}}


While the list of all parameters probed and their corresponding target values is available via the property `bo.res`.


In [11]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -741.1274533629198, 'params': {'a': 2.8340440094051482, 'b': 1.3219469606529488, 'c': -7.9987418770092065, 'd': 1.5116628631591988, 'e': -1.4129764367315478}}
Iteration 1: 
	{'target': -33.65239946885293, 'params': {'a': 2.1846771895375956, 'b': -1.8824387317339746, 'c': -4.1988320025264745, 'd': 1.9838373711533497, 'e': 0.15526693601342778}}
Iteration 2: 
	{'target': -1483.3160846306, 'params': {'a': 2.8383890288065894, 'b': 1.1113170023805568, 'c': -5.751025252953308, 'd': 4.390587181954727, 'e': -1.8904496272082953}}
Iteration 3: 
	{'target': -40.40895345986702, 'params': {'a': 3.3409350203568042, 'b': -0.4961711857972384, 'c': -1.8544118870967319, 'd': 0.7019346929761688, 'e': -1.2075940436604848}}
Iteration 4: 
	{'target': -695.0164547311152, 'params': {'a': 3.6014891373510736, 'b': 2.809569454316385, 'c': -4.552334040248329, 'd': 3.4616130783465704, 'e': 1.5055566091841532}}
Iteration 5: 
	{'target': -55.574013422140546, 'params': {'a': 2.521003392020733

In [12]:
optimizer.set_bounds(new_bounds={'a': (-2, 3)})

In [13]:
optimizer.maximize(
    init_points=0,
    n_iter=5,
)

|   iter    |  target   |     a     |     b     |     c     |     d     |     e     |
-------------------------------------------------------------------------------------
| 9         | -45.42    | -1.865    | -0.9422   | -1.04     | 1.333     | 1.535     |
| 10        | -41.86    | -0.3068   | -0.1481   | 2.471     | 0.6822    | -1.83     |
| 11        | -30.55    | -1.501    | 1.165     | 2.938     | 4.771     | 1.718     |
| 12        | 35.13     | -1.449    | -2.885    | 2.665     | 1.724     | 1.997     |
| 13        | -463.2    | -1.253    | -2.893    | -6.541    | 0.02149   | 1.714     |


## 3. Guiding the optimization

It is often the case that we have an idea of regions of the parameter space where the maximum of our function might lie. For these situations the `BayesianOptimization` object allows the user to specify specific points to be probed. By default these will be explored lazily (`lazy=True`), meaning these points will be evaluated only the next time you call `maximize`. This probing process happens before the gaussian process takes over.

Parameters can be passed as dictionaries such as below:

In [14]:
optimizer.probe(
    params={"a":1, "b":-1, "c":-5, "d":3, "e":0},
    lazy=True,
)

In [15]:
print(optimizer.space.keys)

['a', 'b', 'c', 'd', 'e']


In [16]:
optimizer.probe(
    params=[1, -1, -5, 3, 0],
    lazy=True,
)

In [17]:
optimizer.maximize(
    init_points=0,
    n_iter=0,
)

|   iter    |  target   |     a     |     b     |     c     |     d     |     e     |
-------------------------------------------------------------------------------------
| 14        | -3.0      | 1.0       | -1.0      | -5.0      | 3.0       | 0.0       |
| 15        | -3.0      | 1.0       | -1.0      | -5.0      | 3.0       | 0.0       |


## 2. Getting Started

All we need to get started is to instantiate a `BayesianOptimization` object specifying a function to be optimized `f`, and its parameters with their corresponding bounds, `pbounds`. This is a constrained optimization technique, so you must specify the minimum and maximum values that can be probed for each parameter in order for it to work

In [18]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

The observer paradigm works by:
1. Instantiating an observer object.
2. Tying the observer object to a particular event fired by an optimizer.

The `BayesianOptimization` object fires a number of internal events during optimization, in particular, every time it probes the function and obtains a new parameter-target combination it will fire an `Events.OPTIMIZATION_STEP` event, which our logger will listen to.

**Caveat:** The logger will not look back at previously probed points.

In [19]:
logger = JSONLogger(path="./logs/output-log.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [20]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     a     |     b     |     c     |     d     |     e     |
-------------------------------------------------------------------------------------
| 16        | -709.1    | 2.473     | -2.49     | -7.57     | 0.8492    | 1.513     |
| 17        | 2.313     | -1.508    | -0.4734   | 2.537     | 2.666     | 0.7675    |
| 18        | -454.3    | -0.457    | -2.938    | -3.433    | 4.582     | 1.354     |
| 19        | -221.5    | -0.1408   | -1.262    | -2.872    | 0.9391    | -1.961    |
| 20        | -86.93    | 1.965     | -2.702    | 2.937     | 0.00245   | 0.9418    |


In [21]:
from bayes_opt.util import load_logs

In [22]:
new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2,
    random_state=7,
)
print(len(new_optimizer.space))

0


In [23]:
load_logs(new_optimizer, logs=["./logs/output-log.json"])

/home/tony/miniconda3/envs/bayesian/lib/python3.12/site-packages/bayes_opt/bayesian_optimization.py:223: UserWarning: 
Data point [-1.50826583 -0.47335425  2.53678483  2.66582642  0.76750846] is outside the bounds of the parameter space. 
  self._space.register(params, target, constraint_value)
/home/tony/miniconda3/envs/bayesian/lib/python3.12/site-packages/bayes_opt/bayesian_optimization.py:223: UserWarning: 
Data point [-0.45701499 -2.93819745 -3.43338351  4.58166153  1.3539277 ] is outside the bounds of the parameter space. 
  self._space.register(params, target, constraint_value)
/home/tony/miniconda3/envs/bayesian/lib/python3.12/site-packages/bayes_opt/bayesian_optimization.py:223: UserWarning: 
Data point [-0.14083436 -1.26232533 -2.87175052  0.93911254 -1.96086536] is outside the bounds of the parameter space. 
  self._space.register(params, target, constraint_value)
/home/tony/miniconda3/envs/bayesian/lib/python3.12/site-packages/bayes_opt/bayesian_optimization.py:223: UserWar

In [24]:
print("New optimizer is now aware of {} points of data.".format(len(new_optimizer.space)))

New optimizer is now aware of 5 points of data.


In [25]:
new_optimizer.maximize(
    init_points=0,
    n_iter=10,
)

|   iter    |  target   |     a     |     b     |     c     |     d     |     e     |
-------------------------------------------------------------------------------------
| 1         | -215.0    | 2.0       | 3.0       | 3.0       | 0.0       | -2.0      |
| 2         | -164.1    | 3.166     | -2.893    | 2.576     | 4.982     | -1.598    |
| 3         | -112.2    | 2.205     | 2.95      | 2.493     | 4.825     | 1.296     |
| 4         | -48.46    | 2.681     | -1.126    | 2.978     | 4.512     | 1.952     |
| 5         | -88.41    | 2.61      | 1.068     | 2.844     | 0.5698    | 1.936     |
| 6         | -835.5    | 2.658     | 2.629     | -2.355    | 4.871     | -1.911    |
| 7         | -115.9    | 3.492     | -2.856    | 0.5362    | 0.05154   | -1.776    |
| 8         | -17.01    | 2.106     | -1.401    | 2.94      | 2.599     | -1.784    |
| 9         | -4.959    | 3.919     | -0.6161   | 2.908     | 2.263     | 0.07636   |
| 10        | -213.7    | 3.526     | -2.713    | 2.87

Use Pandas for conversion between CSV and JSON files

In [26]:
import pandas as pd
def convert_json_to_readable_csv(json_file_path, csv_file_path):

    """
    This function converts a JSON file to a readable CSV file with proper formatting and column names.
    """
    #import JSON as a dataframe
    data = pd.read_json(json_file_path, lines=True)

    #flatten nested JSON fields
    data = pd.json_normalize(data.to_dict(orient="records"))


    ## Below is an optional step to clean up the data and export it as a CSV file
    # Rename columns for readability
    data.rename(columns={
        'target': 'Target Value',
        'params.a': 'A',
        'params.b': 'B',
        'params.c': 'C',
        'params.d': 'D',
        'params.e': 'E',
        'datetime.datetime': 'Timestamp',
        'datetime.elapsed': 'Elapsed Time (s)',
        'datetime.delta': 'Time Delta (s)'
    }, inplace=True)

    # Format the Timestamp to a more readable format if necessary
    data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')

    # Example to round numeric values to improve readability
    data['Target Value'] = data['Target Value'].round(2)
    data['A'] = data['A'].round(2)
    data['B'] = data['B'].round(2)
    data['C'] = data['C'].round(2)
    data['D'] = data['D'].round(2)
    data['E'] = data['E'].round(2)
    data['Elapsed Time (s)'] = data['Elapsed Time (s)'].round(3)
    data['Time Delta (s)'] = data['Time Delta (s)'].round(3)

    #save dataframe to CSV
    data.to_csv(csv_file_path, index=False)

    print("JSON Log data has been exported as a CSV file to: {}".format(csv_file_path))



In [27]:
convert_json_to_readable_csv("./logs/output-log.json", "./logs/output-log.csv")

JSON Log data has been exported as a CSV file to: ./logs/output-log.csv


In [ ]:
#TODO: Add conversion of JSON to CSV file
#TODO: Add code to plot the data (aquisition function file has instructions)
#TODO: Add code to save the plot as an image file
#TODO: Add code to save the plot as an interactive HTML file
